Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
from math import sqrt

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Logisting model with regularization

In [5]:
batch_size = 128
beta = 0.003
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss1 = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = loss1 + beta * tf.nn.l2_loss(weights)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 25.521521
Minibatch accuracy: 7.0%
Validation accuracy: 10.6%
Minibatch loss at step 500: 2.321311
Minibatch accuracy: 80.5%
Validation accuracy: 78.1%
Minibatch loss at step 1000: 1.089201
Minibatch accuracy: 79.7%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 0.550351
Minibatch accuracy: 85.2%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 0.598151
Minibatch accuracy: 88.3%
Validation accuracy: 81.7%
Minibatch loss at step 2500: 0.723761
Minibatch accuracy: 80.5%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 0.720549
Minibatch accuracy: 83.6%
Validation accuracy: 81.8%
Test accuracy: 89.1%


Neural Network model with regularization

In [31]:
batch_size = 128
hidden_nodes = 1024
beta = 0.002

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  
  # Training computation.
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), weights2) + biases2
  loss1 = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = loss1 + beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [32]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 975.782532
Minibatch accuracy: 13.3%
Validation accuracy: 25.4%
Minibatch loss at step 500: 232.891296
Minibatch accuracy: 84.4%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 83.688728
Minibatch accuracy: 77.3%
Validation accuracy: 83.0%
Minibatch loss at step 1500: 30.662775
Minibatch accuracy: 91.4%
Validation accuracy: 85.8%
Minibatch loss at step 2000: 11.556829
Minibatch accuracy: 91.4%
Validation accuracy: 86.3%
Minibatch loss at step 2500: 4.629396
Minibatch accuracy: 87.5%
Validation accuracy: 86.4%
Minibatch loss at step 3000: 2.123749
Minibatch accuracy: 85.9%
Validation accuracy: 86.5%
Test accuracy: 92.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [108]:
batch_size = 128

restriction = batch_size*20
train_dataset1 = train_dataset[1:restriction,:]
train_labels1 = train_labels[1:restriction,:]


hidden_nodes = 1024
beta = 0.002

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  
  # Training computation.
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), weights2) + biases2
  loss1 = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = loss1 + beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [109]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels1.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset1[offset:(offset + batch_size), :]
    batch_labels = train_labels1[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 983.466614
Minibatch accuracy: 7.8%
Validation accuracy: 31.2%
Minibatch loss at step 500: 230.810089
Minibatch accuracy: 100.0%
Validation accuracy: 79.4%
Minibatch loss at step 1000: 84.870705
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 1500: 31.211409
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Minibatch loss at step 2000: 11.527271
Minibatch accuracy: 100.0%
Validation accuracy: 80.7%
Minibatch loss at step 2500: 4.337866
Minibatch accuracy: 100.0%
Validation accuracy: 81.9%
Minibatch loss at step 3000: 1.712929
Minibatch accuracy: 100.0%
Validation accuracy: 82.3%
Test accuracy: 89.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [138]:
batch_size = 128

restriction = batch_size*20
train_dataset1 = train_dataset[1:restriction,:]
train_labels1 = train_labels[1:restriction,:]


hidden_nodes = 1024
beta = 0.002

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  
  # Training computation.
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), weights2) + biases2
  loss1 = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = loss1 + beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction1 = tf.nn.softmax(logits)
  #drop out some of the predictions
  train_prediction = tf.nn.dropout(2*train_prediction1,0.5)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [139]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels1.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset1[offset:(offset + batch_size), :]
    batch_labels = train_labels1[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
 
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1022.617920
Minibatch loss at step 0: 1022.617920
Minibatch accuracy: 8.6%
Validation accuracy: 27.6%
Minibatch loss at step 1: 1660.567627
Minibatch loss at step 2: 2231.439209
Minibatch loss at step 3: 1715.567627
Minibatch loss at step 4: 1122.169434
Minibatch loss at step 5: 851.236450
Minibatch loss at step 6: 882.725098
Minibatch loss at step 7: 801.164917
Minibatch loss at step 8: 850.146362
Minibatch loss at step 9: 723.214294
Minibatch loss at step 10: 721.382812
Minibatch loss at step 11: 702.536499
Minibatch loss at step 12: 655.409180
Minibatch loss at step 13: 687.597107
Minibatch loss at step 14: 685.584045
Minibatch loss at step 15: 679.872131
Minibatch loss at step 16: 696.503296
Minibatch loss at step 17: 683.567383
Minibatch loss at step 18: 679.865967
Minibatch loss at step 19: 674.079895
Minibatch loss at step 20: 662.889832
Minibatch loss at step 21: 626.146973
Minibatch loss at step 22: 630.414001
Minibatch loss at step 23: 65

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

In [7]:
batch_size = 128


hidden_nodes1 = 1024
hidden_nodes2 = 512
hidden_nodes3 = 128
beta = 0.0002

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 =  tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes1], 
stddev=sqrt(2.0/(image_size * image_size * hidden_nodes1))))
  weights2 = tf.Variable(tf.truncated_normal([hidden_nodes1, hidden_nodes2], 
stddev=sqrt(2.0/(hidden_nodes1 * hidden_nodes2))))
  weights3 = tf.Variable(tf.truncated_normal([hidden_nodes2, hidden_nodes3], 
stddev=sqrt(2.0/(hidden_nodes2 * hidden_nodes3))))
  weights4 = tf.Variable(tf.truncated_normal([hidden_nodes3, num_labels], 
stddev=sqrt(2.0/(hidden_nodes3 * num_labels))))
  biases1 = tf.Variable(tf.zeros([hidden_nodes1]))
  biases2 = tf.Variable(tf.zeros([hidden_nodes2]))
  biases3 = tf.Variable(tf.zeros([hidden_nodes3]))
  biases4 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  logits2 = tf.matmul(tf.nn.relu(logits1), weights2) + biases2
  logits3 = tf.matmul(tf.nn.relu(logits2), weights3) + biases3
  logits = tf.matmul(logits3, weights4) + biases4
  loss1 = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = loss1 + beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3) + tf.nn.l2_loss(weights4))
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.8, global_step, 500 ,0.96, staircase = True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  #drop out some of the predictions
  #train_prediction = tf.nn.dropout(train_prediction1,0.5)

  valid_prediction_p1 = tf.matmul(tf_valid_dataset, weights1) + biases1
  valid_prediction_p2 = tf.matmul(tf.nn.relu(valid_prediction_p1), weights2) + biases2
  valid_prediction_p3 = tf.matmul(tf.nn.relu(valid_prediction_p2), weights3) + biases3
  valid_prediction_p4 = tf.matmul(tf.nn.relu(valid_prediction_p3), weights4) + biases4
  valid_prediction = tf.nn.softmax(valid_prediction_p4)

  test_prediction_p1 = tf.matmul(tf_test_dataset, weights1) + biases1;
  test_prediction_p2 = tf.matmul(tf.nn.relu(test_prediction_p1), weights2) + biases2
  test_prediction_p3 = tf.matmul(tf.nn.relu(test_prediction_p2), weights3) + biases3
  test_prediction_p4 = tf.matmul(tf.nn.relu(test_prediction_p3), weights4) + biases4
  test_prediction = tf.nn.softmax(test_prediction_p4)
 

In [8]:
num_steps = 15001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.303207
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.418674
Minibatch accuracy: 89.1%
Validation accuracy: 83.3%
Minibatch loss at step 1000: 0.520167
Minibatch accuracy: 85.2%
Validation accuracy: 84.8%
Minibatch loss at step 1500: 0.319967
Minibatch accuracy: 91.4%
Validation accuracy: 86.5%
Minibatch loss at step 2000: 0.326633
Minibatch accuracy: 91.4%
Validation accuracy: 87.4%
Minibatch loss at step 2500: 0.372447
Minibatch accuracy: 89.8%
Validation accuracy: 87.6%
Minibatch loss at step 3000: 0.433887
Minibatch accuracy: 87.5%
Validation accuracy: 87.7%
Minibatch loss at step 3500: 0.444951
Minibatch accuracy: 88.3%
Validation accuracy: 88.4%
Minibatch loss at step 4000: 0.325784
Minibatch accuracy: 92.2%
Validation accuracy: 88.6%
Minibatch loss at step 4500: 0.306234
Minibatch accuracy: 91.4%
Validation accuracy: 87.7%
Minibatch loss at step 5000: 0.422087
Minibatch accuracy: 89.8%
Validation accurac

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---